<a href="https://colab.research.google.com/github/Akeab-tame/Week-08-9_Fraud_Detection/blob/main/Neural_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, GRU, Dropout
import mlflow
import mlflow.tensorflow

In [4]:
# Load preprocessed data
pre_fraud_data_df = pd.read_csv('preprocessed_fraud_data.csv')

In [5]:
pre_fraud_data_df

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,hour_of_day,day_of_week
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,89215,2,0,1,39,732758368,0,84,1,0.000008,2.0,5.0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,24078,0,0,0,53,350311387,0,171,1,0.000892,1.0,0.0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,131216,2,3,1,53,2621473820,1,171,1,15.000000,18.0,3.0
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,3977,2,4,1,41,3840542443,0,181,1,0.000089,13.0,0.0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,68757,0,4,1,45,415583117,0,171,1,0.000009,18.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95022,186435,2015-07-09 23:28:28,2015-07-18 15:20:29,13,93837,2,0,0,53,1628986394,0,171,1,0.000017,15.0,5.0
95023,46764,2015-08-16 13:58:29,2015-11-13 17:25:29,46,19354,1,0,0,27,2803216335,0,171,1,0.000006,17.0,4.0
95024,313982,2015-04-01 19:58:36,2015-05-26 14:09:30,29,135083,1,2,1,25,4169551904,0,181,1,0.000006,14.0,1.0
95025,223769,2015-01-07 09:14:08,2015-01-07 09:14:09,46,11667,2,4,1,20,3094802493,1,171,1,46.000000,9.0,2.0


In [6]:
#Function to extract datetime features month and year
def extract_datetime_features(df, datetime_column):
    df_copy = df.copy()
    df_copy[datetime_column] = pd.to_datetime(df_copy[datetime_column])
    df_copy['year'] = df_copy[datetime_column].dt.year
    df_copy['month'] = df_copy[datetime_column].dt.month
    df_copy = df_copy.drop(columns=[datetime_column])
    return df_copy
datetime_column = 'purchase_time'
fraud_data_df = extract_datetime_features(pre_fraud_data_df, datetime_column)

In [7]:
fraud_data_df

,user_id,signup_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,hour_of_day,day_of_week,year,month
0,22058,2015-02-24 22:55:49,34,89215,2,0,1,39,732758368,0,84,1,0.000008,2.0,5.0,2015,4
1,333320,2015-06-07 20:39:50,16,24078,0,0,0,53,350311387,0,171,1,0.000892,1.0,0.0,2015,6
2,1359,2015-01-01 18:52:44,15,131216,2,3,1,53,2621473820,1,171,1,15.000000,18.0,3.0,2015,1
3,150084,2015-04-28 21:13:25,44,3977,2,4,1,41,3840542443,0,181,1,0.000089,13.0,0.0,2015,5
4,221365,2015-07-21 07:09:52,39,68757,0,4,1,45,415583117,0,171,1,0.000009,18.0,2.0,2015,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95022,186435,2015-07-09 23:28:28,13,93837,2,0,0,53,1628986394,0,171,1,0.000017,15.0,5.0,2015,7
95023,46764,2015-08-16 13:58:29,46,19354,1,0,0,27,2803216335,0,171,1,0.000006,17.0,4.0,2015,11
95024,313982,2015-04-01 19:58:36,29,135083,1,2,1,25,4169551904,0,181,1,0.000006,14.0,1.0,2015,5
95025,223769,2015-01-07 09:14:08,46,11667,2,4,1,20,3094802493,1,171,1,46.000000,9.0,2.0,2015,1


In [8]:
fraud_data_df .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95027 entries, 0 to 95026
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                95027 non-null  int64  
 1   signup_time            95027 non-null  object 
 2   purchase_value         95027 non-null  int64  
 3   device_id              95027 non-null  int64  
 4   source                 95027 non-null  int64  
 5   browser                95027 non-null  int64  
 6   sex                    95027 non-null  int64  
 7   age                    95027 non-null  int64  
 8   ip_address             95027 non-null  int64  
 9   class                  95027 non-null  int64  
 10  country                95027 non-null  int64  
 11  transaction_frequency  95027 non-null  int64  
 12  transaction_velocity   95027 non-null  float64
 13  hour_of_day            95026 non-null  float64
 14  day_of_week            95026 non-null  float64
 15  ye

In [9]:
# Convert 'year' and 'month' columns to int64
fraud_data_df['year'] = fraud_data_df['year'].astype('int64')
fraud_data_df['month'] = fraud_data_df['month'].astype('int64')
fraud_data_df['transaction_velocity'] = fraud_data_df['transaction_velocity'].astype('int64')

In [10]:
fraud_data_df

,user_id,signup_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,hour_of_day,day_of_week,year,month
0,22058,2015-02-24 22:55:49,34,89215,2,0,1,39,732758368,0,84,1,0,2.0,5.0,2015,4
1,333320,2015-06-07 20:39:50,16,24078,0,0,0,53,350311387,0,171,1,0,1.0,0.0,2015,6
2,1359,2015-01-01 18:52:44,15,131216,2,3,1,53,2621473820,1,171,1,15,18.0,3.0,2015,1
3,150084,2015-04-28 21:13:25,44,3977,2,4,1,41,3840542443,0,181,1,0,13.0,0.0,2015,5
4,221365,2015-07-21 07:09:52,39,68757,0,4,1,45,415583117,0,171,1,0,18.0,2.0,2015,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95022,186435,2015-07-09 23:28:28,13,93837,2,0,0,53,1628986394,0,171,1,0,15.0,5.0,2015,7
95023,46764,2015-08-16 13:58:29,46,19354,1,0,0,27,2803216335,0,171,1,0,17.0,4.0,2015,11
95024,313982,2015-04-01 19:58:36,29,135083,1,2,1,25,4169551904,0,181,1,0,14.0,1.0,2015,5
95025,223769,2015-01-07 09:14:08,46,11667,2,4,1,20,3094802493,1,171,1,46,9.0,2.0,2015,1


In [12]:
# Encode Categorical Features
def encode_categorical_features(df):
    le = LabelEncoder()
    for column in df.select_dtypes(include=['object']).columns:
        df[column] = le.fit_transform(df[column])
    return df
fraud_data_df = encode_categorical_features(fraud_data_df)

In [13]:
fraud_data_df

,user_id,signup_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,hour_of_day,day_of_week,year,month
0,22058,26356,34,89215,2,0,1,39,732758368,0,84,1,0,2.0,5.0,2015,4
1,333320,66995,16,24078,0,0,0,53,350311387,0,171,1,0,1.0,0.0,2015,6
2,1359,598,15,131216,2,3,1,53,2621473820,1,171,1,15,18.0,3.0,2015,1
3,150084,51191,44,3977,2,4,1,41,3840542443,0,181,1,0,13.0,0.0,2015,5
4,221365,84158,39,68757,0,4,1,45,415583117,0,171,1,0,18.0,2.0,2015,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95022,186435,79603,13,93837,2,0,0,53,1628986394,0,171,1,0,15.0,5.0,2015,7
95023,46764,94382,46,19354,1,0,0,27,2803216335,0,171,1,0,17.0,4.0,2015,11
95024,313982,40536,29,135083,1,2,1,25,4169551904,0,181,1,0,14.0,1.0,2015,5
95025,223769,4967,46,11667,2,4,1,20,3094802493,1,171,1,46,9.0,2.0,2015,1


In [14]:
# Save preprocessed datasets
fraud_data_df.to_csv('cleaned_Fraud_Data.csv', index=False)